# Denison DA210/CS181 SW Lab #10 - Step 1

Before you get your checkpoints, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells**.

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

#### Import Python modules and load "SQL Magic"

In [ ]:
import pandas as pd
import os
import os.path
import json
import sys
import importlib

module_dir = "../../modules"
module_path = os.path.abspath(module_dir)
if not module_path in sys.path:
    sys.path.append(module_path)

%load_ext sql

In [ ]:
def getsqlite_creds(dirname=".",filename="creds.json"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D["sqlite1"]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [ ]:
scheme, dbdir, database = getsqlite_creds()
template = '{}:///{}/{}.db'
cstring = template.format(scheme, dbdir, database)
print("Connection string:", cstring)

#### Establish Connection from Client to Server

In [ ]:
%sql $cstring

---

# Part A: SQL Grammar (Recap)

So far, we have seen the following SQL grammar:

```
    SELECT field-spec
    FROM table-spec
    [order-clause]
    [limit-clause]
```

For example, we may want to project the `pop` and `gdp` columns from `indicators`, but only for the country/year combinations with the 5 highest life expectancies:

```SQL
    SELECT pop AS Population, gdp AS GDP
    FROM indicators
    ORDER BY life DESC
    LIMIT 5
```

We can execute this command using a single-line SQL statement, as follows:

In [ ]:
%sql SELECT pop AS Population, gdp AS GDP FROM indicators ORDER BY life DESC LIMIT 5

That's pretty hard to read, so we can instead build our SQL query as a multi-line Python string, and pass that string to the "SQL magic":

In [ ]:
query = """
SELECT pop AS Population, gdp AS GDP
FROM indicators
ORDER BY life DESC
LIMIT 5
"""

%sql $query

---

# Part B: SQL Single-Table Selection and Row Filtering

We can expand on the multi-column projection queries we've looked at so far to include filtering based on the values in a given field (column).

As a simple example, we can select only the distinct values of a given field.  To do so, we'll add an optional `DISTINCT` keyword to our grammar (recall that `[]` indicate that the contents are optional):
```
    SELECT [DISTINCT] field-spec
    FROM table-spec
    [order-clause]
    [limit-clause]
```

For example, we may want the unique country codes from the `indicators` table in the `book` database:

In [ ]:
query = """
SELECT DISTINCT code
FROM indicators
"""

# Get the results as a DataFrame so we don't have to view the whole thing (it's long)
resultset = %sql $query
resultdf = resultset.DataFrame()
resultdf

Alternatively, we could filter based on a filter condition.  For this, we'll add one new clause to our grammar:
```
    SELECT [DISTINCT] field-spec
    FROM table-spec
    [WHERE [NOT] filter-cond]
    [order-clause]
    [limit-clause]
```

We will look at several types of filter conditions:
1. Binary operations and Booleans
2. Range of values
3. In a set
4. Like (matching) a pattern
5. Null/not null

### 1. Binary operations/Booleans

Filter conditions using binary operations and Booleans have the form `expr op expr`, using the following possible choices for `op`: `=`, `<>`, `<`, `>`, `<=`, `>=`, `NOT`, `AND`, `OR`.

For example:

In [ ]:
query = """
SELECT code, pop
FROM indicators
WHERE gdp > 5000 AND year = 2012
"""

%sql $query

In [ ]:
query = """
SELECT name, count
FROM topnames
WHERE year >= 2015 AND year < 2018 AND sex <> 'Female'
"""

%sql $query

### 2. Range of values

The last example above could be simplified by instead specifying that we want the year between `2015` and `2017`.  We can do that using a range clause:

```
    range-clause |= expr [NOT] BETWEEN low AND high
```

For example:

In [ ]:
query = """
SELECT name, count
FROM topnames
WHERE (year BETWEEN 2015 AND 2017) AND sex <> 'Female'
"""

%sql $query

### 3. In a set

If we instead want to find the records for which a given value is one of a set, we can use an `IN` clause:

```
    in-clause |= expr [NOT] IN (set-members)
```

For example:

In [ ]:
query = """
SELECT code, pop
FROM indicators
WHERE year = 2017 AND code IN ('VNM', 'KOR')
"""

%sql $query

### 4. Like (matching) a pattern

If we want the records for which a given value matches part of a string pattern, we can use a `LIKE` clause:

```
    like-clause |= expr [NOT] LIKE pattern
```

With a `LIKE` clause, we can use either `%` to match multiple characters or `_` to match a single character.

For example:

In [ ]:
query = """
SELECT code, country, region
FROM countries
WHERE country LIKE '%Republic' OR code LIKE '_ZA'
"""

%sql $query

### 5. Null/not null

If we want the records for which a given value is or is not missing, we can use a `NULL` clause:

```
    null-clause |= expr IS [NOT] NULL
```

For example:

In [ ]:
query = """
SELECT code, country, land
FROM countries
WHERE land IS NULL
"""

%sql $query

---

## Part C: Try it yourself

In the following cells, your only action is to remove the `# YOUR CODE HERE` and `raise NotImplementedError()` lines, and then put a valid SQL statement as the **value** of string variable `query`.

In each case, when you execute the cell, the query will be sent to the database management system, a result obtained, and the result converted into a `pandas` data frame, whose prefix is shown.  (This allows for testing the results as well as displaying them in your Jupyter Notebook.)

_Note: you may want to view the `book` database in SQLiteStudio to get an idea of what the field names are as you work through these exercises.  Alternatively, start by adding a few code cells to perform a `SELECT * FROM tablename` query for each table name._

**Q1:** Using the table `indicators`, write a query to find all unique years that appear in the table.

In [ ]:
query = """
"""
# YOUR CODE HERE
raise NotImplementedError()

resultset1 = %sql $query
resultdf1 = resultset1.DataFrame()
resultdf1.head()

In [ ]:
# Testing cell
assert len(resultdf1) == 59

**Q2:** Using the table `indicators`, write a query to find all rows (you can select all fields for these rows) with no missing data for `gdp`.

In [ ]:
query = """
"""
# YOUR CODE HERE
raise NotImplementedError()

resultset2 = %sql $query
resultdf2 = resultset2.DataFrame()
resultdf2.head()

In [ ]:
# Testing cell
assert len(resultdf2) == 9660

**Q3:** Using the table `indicators`, write a query to find all rows with no missing data for any of the numeric fields.

In [ ]:
query = """
"""
# YOUR CODE HERE
raise NotImplementedError()

resultset3 = %sql $query
resultdf3 = resultset3.DataFrame()
resultdf3.head()

In [ ]:
# Testing cell
assert len(resultdf3) == 6958

**Q4:** Using the table `indicators`, write a query to find all rows where exports is higher than imports. Select all fields for such rows.

In [ ]:
query = """
"""
# YOUR CODE HERE
raise NotImplementedError()

resultset4 = %sql $query
resultdf4 = resultset4.DataFrame()
resultdf4.head()

In [ ]:
# Testing cell
assert len(resultdf4) == 2492
assert resultdf4.shape == (2492,8)

**Q5:** Using the table `indicators`, write a query to find the minimum non-zero number for `cell` that appears.  Although you could use `MIN` (which we haven't seen yet), you should try to do this using `ORDER BY` instead, so that the first row in your result has that minimum value.

In [ ]:
query = """
"""
# YOUR CODE HERE
raise NotImplementedError()

resultset5 = %sql $query
resultdf5 = resultset5.DataFrame()
resultdf5.head()

In [ ]:
# Testing cell
assert resultdf5.loc[0,'cell'] == 0.01

> You've reached the first checkpoint in the lab.  Make sure to have it signed off by the instructor or TA.
>
> Checkpoint 1: Convert the following `pandas` selection into a SQL query:
>     `topnames.loc[(1961, 'Female'), 'count']`

---

# Part D: Subqueries

We can use the result of a query as a building block in another query.  For example, we may want to know which countries have _ever_ had a GDP greater than `3000`:

In [ ]:
query = """
SELECT DISTINCT code
FROM indicators
WHERE gdp > 3000
"""

%sql $query

Then, four just these five countries, we can look up their `2017` population and life expectancy values:

In [ ]:
query = """
SELECT code, pop, life
FROM indicators
WHERE year = 2017 AND code IN ('USA', 'JPN', 'DEU', 'CHN', 'GBR')
"""

%sql $query

However, we don't always want to hard-code these country codes.  We can instead use the results of the first query as the `set-members` for the second query:

In [ ]:
query = """
SELECT code, pop, life
FROM indicators
WHERE year = 2017 AND code IN (SELECT DISTINCT code FROM indicators WHERE gdp > 3000)
"""

%sql $query

As another example, we can query the most popular male baby name from 2016:

In [ ]:
query = """
SELECT name
FROM topnames
WHERE year = 2016 AND sex = 'Male'
"""

%sql $query

Then, we can use this result to find all years for which `Noah` was the most popular male baby name, and the associated counts:

In [ ]:
query = """
SELECT year, count
FROM topnames
WHERE name = (SELECT name FROM topnames WHERE year = 2016 AND sex = 'Male')
"""

%sql $query

Similarly, we can use a subquery as our table, rather than as part of a `WHERE` clause:

In [ ]:
query = """
SELECT *
FROM countries
WHERE land < 100
"""

%sql $query

In [ ]:
query = """
SELECT country AS Name, land AS Land
FROM (SELECT * FROM countries WHERE land < 100)
WHERE income = 'High income'
"""

%sql $query

**Q6:** Use a subquery to select the top ten entries in `indicators` with the highest population, then select all fields for the three from that group of ten that have the lowest GDP.

_Hint: You may find it helpful to first write the query just for the top ten entries in `indicators` with the highest population._

In [ ]:
query = """
"""
# YOUR CODE HERE
raise NotImplementedError()

resultset6 = %sql $query
resultdf6 = resultset6.DataFrame()
resultdf6.head()

In [ ]:
# Testing cell
assert len(resultdf6) == 3
assert resultdf6.loc[0,'gdp'] == 2652.55
assert resultdf6.loc[1,'pop'] == 1352.62

> You've reached the second (and final) checkpoint in the lab.  Make sure to have it signed off by the instructor or TA.
>
> Checkpoint 2: Consider the following query from above:
> ```
>     SELECT code, pop, life
>     FROM indicators
>     WHERE year = 2017 AND code IN
>         (SELECT DISTINCT code FROM indicators WHERE gdp > 3000)
> ```
> How is this different from just looking for `gdp > 3000` within the `WHERE` clause, as below:
> ```
>     SELECT code, pop, life
>     FROM indicators
>     WHERE year = 2017 AND gdp > 3000
> ```

---

---
## Part E

How much time (in minutes/hours) did you spend on this lab outside of class?

YOUR ANSWER HERE